In [3]:
import sqlalchemy
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv


In [6]:
load_dotenv()
user = os.getenv('user')
password = os.getenv('password')
host = os.getenv('host')
db = os.getenv('db')
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:5439/{db}')

In [7]:
### using in memory db
engine = create_engine('sqlite:///:memory:', echo=True)

In [8]:
### declaring a mapping 
from sqlalchemy.orm import declarative_base
Base = declarative_base()

In [9]:
## creating a mapped class for a single table
from sqlalchemy import Column, Integer, String
class User(Base): # inherits from declarative base
    __tablename__ = 'users'
    # defining the table properties and attributes -> primary_key column is necessary

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)


    def __repr__(self) -> str: # defines the format of our output
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name,
            self.fullname,
            self.nickname
        )

In [11]:
## inspecting our table metadata
User.__table__

Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [13]:
# using the Metadata registry to create a table
Base.metadata.create_all(engine)

2023-03-17 21:41:32,586 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 21:41:32,595 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2023-03-17 21:41:32,597 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-17 21:41:32,603 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2023-03-17 21:41:32,604 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-17 21:41:32,610 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2023-03-17 21:41:32,613 INFO sqlalchemy.engine.Engine [no key 0.00233s] ()
2023-03-17 21:41:32,615 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
## for firebird and Oracle sequence is required to generate new PK

from sqlalchemy import Sequence

Column(Integer, Sequence("user_id_seq"), primary_key=True)

class User(Base):
    __tablename__ = "users"
    id = Column(Integer, Sequence("user_id_seq"), primary_key=True)
    name = Column(String(50))
    fullname = Column(String(50))
    nickname = Column(String(50))

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name,
            self.fullname,
            self.nickname,
        )


In [14]:
# creating instance of the mapped class User
ed_user = User(name='ed', fullname='Éd Jones', nickname= 'edsnickname')
ed_user.name

'ed'

Note the Base declarative system automatically provided the __init__() constructor class this can be declared explicitly which will over ride the default provided by Base

In [15]:
# creating a session this allows us to speak to the db
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)


In [16]:
session = Session()

In [17]:
# adding objects to our User
ed_user = User(name='ed', fullname='Éd Jones', nickname= 'edsnickname')
session.add(ed_user)

In [18]:
# Getting our user
our_user = (
    session.query(User).filter_by(name='ed').first()
)

2023-03-17 21:57:28,960 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 21:57:28,973 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2023-03-17 21:57:28,974 INFO sqlalchemy.engine.Engine [generated in 0.00183s] ('ed', 'Éd Jones', 'edsnickname')
2023-03-17 21:57:28,993 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2023-03-17 21:57:28,994 INFO sqlalchemy.engine.Engine [generated in 0.00091s] ('ed', 1, 0)


In [19]:
our_user

<User(name='ed', fullname='Éd Jones', nickname='edsnickname')>

In [20]:
ed_user is our_user

True

In [21]:
# use add_all() to add more than one User object at once

session.add_all(
    [
        User(name="wendy", fullname="Wendy Williams", nickname="windy"),
        User(name="mary", fullname="Mary Contrary", nickname="mary"),
        User(name="fred", fullname="Fred Flintstone", nickname="freddy"),
    ]
)

In [22]:
# we can also change objects attribute simply by reassigning them

ed_user.nickname = 'eddie'

In [23]:
session.dirty

IdentitySet([<User(name='ed', fullname='Éd Jones', nickname='eddie')>])

In [24]:
session.new # displays pending commits to be made

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

In [25]:
# to add changes to the db we need to commit

session.commit()

2023-03-17 22:27:14,387 INFO sqlalchemy.engine.Engine UPDATE users SET nickname=? WHERE users.id = ?
2023-03-17 22:27:14,395 INFO sqlalchemy.engine.Engine [generated in 0.01146s] ('eddie', 1)
2023-03-17 22:27:14,411 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?) RETURNING id
2023-03-17 22:27:14,412 INFO sqlalchemy.engine.Engine [generated in 0.00036s (insertmanyvalues)] ('wendy', 'Wendy Williams', 'windy', 'mary', 'Mary Contrary', 'mary', 'fred', 'Fred Flintstone', 'freddy')
2023-03-17 22:27:14,419 INFO sqlalchemy.engine.Engine COMMIT


In [26]:
ed_user.id

2023-03-17 22:44:26,272 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 22:44:26,290 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2023-03-17 22:44:26,291 INFO sqlalchemy.engine.Engine [generated in 0.00424s] (1,)


1